# Residual Vector Quantization
 Residual Vector Quantization (RVQ) is a hierarchical vector quantization technique where a signal is approximated using multiple stages of quantization, with each stage refining the residual error left by the previous stage. It is particularly useful for efficient data compression, feature representation, and neural network quantization.

In [2]:
! pip install numpy

  Using cached numpy-2.2.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.3-cp313-cp313-win_amd64.whl (12.6 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np

# RVQ Class
The class initailizes an RVQ system with:
- num_stages : Number of refinement steps.
- codebook_size : Number of vectors in each stage's codebook.
- vector_dim : Dimensionality of the vectors.

Random codebooks are generated for each stage, using np.random.radn()

In [8]:
class ResidualVectorQuantizer:
    def __init__(self, num_stages=3, codebook_size=8, vector_dim=2):
        self.num_stages = num_stages
        self.codebook_size = codebook_size
        self.vector_dim = vector_dim
        self.codebooks = [np.random.randn(codebook_size,vector_dim) for _ in range(num_stages)]

# Quantizing the Input Vector
1. Initialize the residual as the input vector x.
2. Loop through each stage of refinement:
    - Find the best-matching codebook vector by computing the Euclidean distance.
    - Store the index (best_match_idx) and subtract the quantized vector from the residual.
3. Return the quantized vectors and their corresponding indices.


In [15]:
class ResidualVectorQuantizer:
    def __init__(self, num_stages=3, codebook_size=8, vector_dim=2):
        self.num_stages = num_stages
        self.codebook_size = codebook_size
        self.vector_dim = vector_dim
        self.codebooks = [np.random.randn(codebook_size,vector_dim) for _ in range(num_stages)]
        
    def quantize(self, x):
        residual = x
        quantized_vectors =[]
        indices = []
        
        for stage in range(self.num_stages):
            codebook = self.codebooks[stage]
            distances = np.linalg.norm(residual - codebook, axis=1)
            best_match_idx = np.argmin(distances, axis=0)

            quantized_vector = codebook[best_match_idx]
            residual -= quantized_vector

            quantized_vectors.append(quantized_vector)
            indices.append(best_match_idx)

        return quantized_vectors, indices

# Reconstructing the Vector
- The original input vector is reconstructed by summing the selected vectors from each stage.
- This approximates the original input but may have some residual error.

In [20]:
class ResidualVectorQuantizer:
    def __init__(self, num_stages=3, codebook_size=8, vector_dim=2):
        self.num_stages = num_stages
        self.codebook_size = codebook_size
        self.vector_dim = vector_dim
        self.codebooks = [np.random.randn(codebook_size,vector_dim) for _ in range(num_stages)]
        
    def quantize(self, x):
        residual = x
        quantized_vectors =[]
        indices = []
        
        for stage in range(self.num_stages):
            codebook = self.codebooks[stage]
            distances = np.linalg.norm(residual - codebook, axis=1)
            best_match_idx = np.argmin(distances, axis=0)

            quantized_vector = codebook[best_match_idx]
            residual -= quantized_vector

            quantized_vectors.append(quantized_vector)
            indices.append(best_match_idx)

        return quantized_vectors, indices

    def reconstruct(self, indices):
        reconstructed = np.zeros(self.vector_dim)  # Start with zero vector
        for stage in range(self.num_stages):
            reconstructed += self.codebooks[stage][indices[stage]]  # Add each stage's quantized vector
        return reconstructed

# Testing the RVQ Implementation

In [21]:
# Test RVQ
rvq = ResidualVectorQuantizer(num_stages=3, codebook_size=8, vector_dim=2)

# Generate a random input vector
x = np.array([2.5, -1.3]) # Example 2D vector

# Apply quantization
quantized_vectors, indices = rvq.quantize(x)

# Reconstruct the vector
reconstructed_x = rvq.reconstruct(indices)

# Print the results
print(f"Original vector: {x}")
print(f"Reconstructed Vector: {reconstructed_x}")
print(f"Residual Error: {np.linalg.norm(x - reconstructed_x)}")

Original vector: [ 0.24182299 -0.33086157]
Reconstructed Vector: [ 2.25817701 -0.96913843]
Residual Error: 2.114965933504104


NOTE : The residual error is large (Residual_Error>>0) because the codebooks are randomly generated. This is a common issue when using random codebooks. To improve the performance, K-means can be used to train cookbooks.